In [1]:
using Distributions

abstract type Component end
abstract type Scatterer end

In [2]:
mutable struct Parameter
    value::Real
    vary::Bool
    bounds::Distribution
    name::Union{String, Symbol}
    function Parameter(value::Real;
            vary::Bool=false,
            bounds::Distribution=Uniform(-Inf, Inf),
            name::Union{String, Symbol}=""
        )
        new(value, vary, bounds, name)
    end
end

build_parameter(p::Union{Parameter, Real}) = isa(p, Parameter) ? p : Parameter(p)


struct Structure <: AbstractVector{Component}
    components::Vector{Component}
end
Structure() = Structure([])

Base.size(s::Structure) = size(s.components)
Base.getindex(s::Structure, i::Int) = s.components[i]
Base.IndexStyle(::Type{<:Structure}) = IndexLinear()
function Base.push!(s::Structure, items...)
    for item in items
        push!(s.components, item)
    end
end
function Base.append!(s::Structure, arr::Array{N, 1} where N<:Component)
    for item in arr
        push!(s.components, item)
    end
end
Base.setindex!(s::Structure, c::Component, i::Int) = setindex!(s.components, c, i)


mutable struct SLD <: Scatterer
    re::Parameter
    im::Parameter
    SLD(re::Union{Real, Parameter}) = new(build_parameter(re), Parameter(0.0))
    SLD(re::Union{Real, Parameter}, im::Union{Real, Parameter}) = new(
        build_parameter(re), build_parameter(im)
    )
    SLD(sld::Complex) = new(Parameter(sld.re), Parameter(sld.im))
end
sld(s::SLD) = s.re.value + 1.0im * s.im.value


mutable struct Slab <: Component
    thickness::Parameter
    scatterer::Scatterer
    roughness::Parameter
    vfsolv::Parameter
    function Slab(thickness::Union{Parameter, Real},
        scatterer::Union{Scatterer, Real, Complex, Tuple{Real, Real}},
        roughness::Union{Parameter, Real},
        vfsolv::Union{Parameter, Real})
        if isa(scatterer, Scatterer)
            s = scatterer
        else
            s = SLD(scatterer)
        end
        new(build_parameter(thickness), s, build_parameter(roughness), build_parameter(vfsolv)
    )
    end
end
function slabs(slab::Slab)
    sldv = sld(slab.scatterer)
    return [slab.thickness.value, sldv.re, sldv.im, slab.roughness.value]
end


(|)(a::Component, b::Component) = Structure([a, b])
function |(s::Structure, b::Component)
    push!(s, b)
    s
end
function |(s::Structure, arr::Array{N, 1} where N<:Component)
    append!(s, arr)
    s
end

| (generic function with 3 methods)

In [3]:
p = Parameter(2.07; vary=false, name="Si")

Parameter(2.07, false, Uniform{Float64}(a=-Inf, b=Inf), "Si")

In [4]:
si = SLD(p)

SLD(Parameter(2.07, false, Uniform{Float64}(a=-Inf, b=Inf), "Si"), Parameter(0.0, false, Uniform{Float64}(a=-Inf, b=Inf), ""))

In [12]:
s = Slab(100, 2.07, 0, 4)

Slab(Parameter(100, false, Uniform{Float64}(a=-Inf, b=Inf), ""), SLD(Parameter(2.07, false, Uniform{Float64}(a=-Inf, b=Inf), ""), Parameter(0.0, false, Uniform{Float64}(a=-Inf, b=Inf), "")), Parameter(0, false, Uniform{Float64}(a=-Inf, b=Inf), ""), Parameter(4, false, Uniform{Float64}(a=-Inf, b=Inf), ""))

In [6]:
sld(s.scatterer)

2.07 + 0.0im

In [7]:
slabs(s)

4-element Array{Any,1}:
  Parameter(100, false, Uniform{Float64}(a=-Inf, b=Inf), "")
 2.07
 0.0
  Parameter(0, false, Uniform{Float64}(a=-Inf, b=Inf), "")

In [19]:
isa(true, Real)

true

In [20]:
s.thickness.value = "a"

LoadError: MethodError: Cannot `convert` an object of type String to an object of type Real
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [ ]:
s = Slab(1.0, SLD(2.0, 1), 3.0, 4.0)
t = Slab(1.0, si, 3.0, 4.0)
s, t

In [ ]:
Structure([s, t])

In [ ]:
p = s | t | s

In [ ]:
p |= 2

In [ ]:
println(objectid(p))
p = p | t
println(objectid(p))
p

In [ ]:
println(objectid(p))
p |= t
println(objectid(p))
p

In [ ]:
println(objectid(p))
p |= [s, t]
println(objectid(p))
p

In [ ]:
s==t, isequal(s, t), s===t

In [ ]:
s.thickness == t.thickness

In [ ]:
push!(p, s, t)
p

In [ ]:
append!(p, [s, t])
p

In [ ]:
for l in p
    println(l)
end

In [ ]:
eltype(p)

In [ ]:
t in p

In [ ]:
in(t, p)

In [ ]:
p |= [s, t]
p

In [ ]:
p |=  [s, t]
p

In [ ]:
using Distributions

In [ ]:
import Pkg; Pkg.add("Distributions")

In [ ]:
using Distributions

In [ ]:
n = Normal(0, 1)

In [ ]:
pdf(n, 2.0)

In [ ]:
typeof(n)

In [ ]:
isa(n, Distribution)

In [ ]:
ValueSupport

In [ ]:
u = Uniform(-10, 10)

In [ ]:
pdf(u, 0)

In [ ]:
f(x) = isa(x, Real) ? x : "a"

In [ ]:
f("b")